In [1]:
import dash
import pandas as pd
import numpy as np
import plotly.express as px
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [2]:
# Helper function to find special points in the dataset

def find_special_points(value, points):
    if value == points.max():
        special_point_name = "max"
    elif value == points.min():
        special_point_name = "min"
    elif value == points.median():
        special_point_name = "median"
    else:
        special_point_name = "not special"
    return special_point_name

In [3]:
# Initialize app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div(children=[
        # Header section with app title and description
        html.Div(children=[
                html.H2("Random Points Generator", style={"text-align": "center"}),
                html.P(
                    "This application generates random points and visualizes them in a scatter plot. "
                    "Use the dropdown menu to select the number of points and observe how special values (max, min, median) are highlighted.",
                    style={"text-align": "center"},
                ),
            ],
            style={
                "padding": "20px",
                "background-color": "#f7f7f7",
                "border-bottom": "1px solid #ccc", # dividing line
            },
        ),
        # Main content section
        html.Div(children=[
                # Scatter plot
                dcc.Graph(id="graph", style={"flex": "1"}),  
                # Dropdown menu for selecting the number of points
                html.Div(children=[
                        html.P("Choose number of points:", style={"margin-bottom": "10px"}),
                        dcc.Dropdown(
                            id="number-dropdown",
                            clearable=False,
                            value=45,  # Default value
                            options=[{"label": str(num), "value": num} for num in [45, 75, 95]],
                        ),
                    ],
                    style={
                        "padding": "10px",
                        "flex": "1",
                        "text-align": "center",
                    },
                ),
            ],
            style={
                "display": "flex",
                "flex-direction": "row",
                "justify-content": "space-around",
                "align-items": "center",
                "padding": "20px",
                "gap": "20px",  # Space between graph and dropdown
            },
        ),
    ],
    style={
        "display": "flex",
        "flex-direction": "column",
        "height": "100vh",
        "fontFamily": "Verdana",
        "color": "#444",
    },
)

# Define the callback to update the scatter plot based on dropdown selection
@app.callback(
    Output("graph", "figure"),
    [Input("number-dropdown", "value")],
)
def update_figure(points_number):
    # Generate random points
    numbers = pd.DataFrame(np.random.randn(points_number), columns=["sample"])
    numbers["special_points"] = numbers["sample"].apply(lambda x: find_special_points(x, numbers["sample"]))

    # Create the scatter plot
    chart = px.scatter(
        numbers,
        x=numbers.index,
        y="sample",
        color="special_points",
        color_discrete_map={
            "max": "#800E13",
            "median": "#169873",
            "min": "#2C497F",
            "not special": "#E4DFDA",
        },
        render_mode="webgl",
        template="none",
        size=[10 for _ in numbers["sample"]],
        title="Scatter Plot of Random Points",
    )
    return chart

# Run app
if __name__ == "__main__":
    app.run_server(debug=True)